<a href="https://colab.research.google.com/github/arnabnandy7/llm_notebooks/blob/main/L1-Embeddings-api-intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>